<a href="https://colab.research.google.com/github/papichoolo/DSCMentalHealthChatbot/blob/main/Falcon7bMentalHealth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
from huggingface_hub import notebook_login
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [8]:

from torch import cuda, bfloat16
import transformers
model_id_1 = 'vilsonrodrigues/falcon-7b-instruct-sharded'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
model_config = transformers.AutoConfig.from_pretrained(
    model_id_1,
    use_auth_token=hf_auth,
    trust_remote_code=True
)

model_2 = transformers.AutoModelForCausalLM.from_pretrained(
    model_id_1,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',

)
model_2.eval()
print(f"Model loaded on {device}")

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded on cuda:0


In [9]:

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id_1

)

In [10]:
generate_text = transformers.pipeline(
    model=model_2, tokenizer=tokenizer,
      # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'FalconForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'Pegasu

In [11]:
from langchain.llms import HuggingFacePipeline
from langchain.chat_models import ChatOpenAI
llm = HuggingFacePipeline(pipeline=generate_text)

# Creating Prompt


In [12]:
!pip install langchain

In [16]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
template = sys_msg = "<s>" + B_SYS + """
You are a mental health adviser. You will have a conversation with an individual looking for guidance and advice on managing their emotions, stress, anxiety and other mental health issues. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing after listening to your patient like a friend. You will stop after the Human is done talking




Here is the latest conversation between You and User \n
 {chat_history} """ + E_SYS
instruction = B_INST + " You will listen to this User and give mental support. Remember to only be the Mental Health Adviser " + E_INST
human_msg = instruction + "\nUser: {human_input} \n Mental Health Adviser:"
template+=human_msg
prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True)


# Run the chain only specifying the input variable.
#chat_history= chain.predict(human_input="Hi there my friend")
#chat_history= enforce_stop_tokens(chat_history, stop_terms)

In [17]:
for i in range(5):
  h_input= input("Talk to us...")
  print(chain.predict(human_input=h_input))

Talk to us...I am depressed


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
<s><<SYS>>

You are a mental health adviser. You will have a conversation with an individual looking for guidance and advice on managing their emotions, stress, anxiety and other mental health issues. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing after listening to your patient like a friend. You will stop after the Human is done talking




Here is the latest conversation between You and User 

  
<</SYS>>

[INST] You will listen to this User and give mental support. Remember to only be the Mental Health Adviser [/INST]
User: I am depressed 
 Mental Health Adviser:

> Finished chain.
 I understand that you are feeling down right now. It's okay to feel this way sometimes. Have you tried any relaxation techniques or therapy to hel

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
<s><<SYS>>

You are a mental health adviser. You will have a conversation with an individual looking for guidance and advice on managing their emotions, stress, anxiety and other mental health issues. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing after listening to your patient like a friend. You will stop after the Human is done talking




Here is the latest conversation between You and User 

 Human: I am depressed
AI:  I understand that you are feeling down right now. It's okay to feel this way sometimes. Have you tried any relaxation techniques or therapy to help manage your emotions?
User  
<</SYS>>

[INST] You will listen to this User and give mental support. Remember to only be the Mental Health Adviser [/INST]
User: No, 

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
<s><<SYS>>

You are a mental health adviser. You will have a conversation with an individual looking for guidance and advice on managing their emotions, stress, anxiety and other mental health issues. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing after listening to your patient like a friend. You will stop after the Human is done talking




Here is the latest conversation between You and User 

 Human: I am depressed
AI:  I understand that you are feeling down right now. It's okay to feel this way sometimes. Have you tried any relaxation techniques or therapy to help manage your emotions?
User 
Human: No, I havent... how do i even begin
AI: 

It's important to take care of your mental health. There are many ways to manage depres

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
<s><<SYS>>

You are a mental health adviser. You will have a conversation with an individual looking for guidance and advice on managing their emotions, stress, anxiety and other mental health issues. You should use your knowledge of cognitive behavioral therapy, meditation techniques, mindfulness practices, and other therapeutic methods in order to create strategies that the individual can implement in order to improve their overall wellbeing after listening to your patient like a friend. You will stop after the Human is done talking




Here is the latest conversation between You and User 

 Human: I am depressed
AI:  I understand that you are feeling down right now. It's okay to feel this way sometimes. Have you tried any relaxation techniques or therapy to help manage your emotions?
User 
Human: No, I havent... how do i even begin
AI: 

It's important to take care of your mental health. There are many ways to manage depres

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py:851 in raw_input                 │
│                                                                                                  │
│   848 │   │   │   raise StdinNotImplementedError(                                                │
│   849 │   │   │   │   "raw_input was called, but this frontend does not support input requests   │
│   850 │   │   │   )                                                                              │
│ ❱ 851 │   │   return self._input_request(str(prompt),                                            │
│   852 │   │   │   self._parent_ident,                                                            │
│   853 │   │   │   self._parent_header,                                                           │
│   854 │   │   │   password=False,                                                                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py:895 in _input_request            │
│                                                                                                  │
│   892 │   │   │   │   │   │   break                                                              │
│   893 │   │   │   except KeyboardInterrupt:                                                      │
│   894 │   │   │   │   # re-raise KeyboardInterrupt, to truncate traceback                        │
│ ❱ 895 │   │   │   │   raise KeyboardInterrupt("Interrupted by user") from None                   │
│   896 │   │   │   except Exception as e:                                                         │
│   897 │   │   │   │   self.log.warning("Invalid Message:", exc_info=True)                        │
│   898                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt: Interrupted by user

# Test original model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Prep dataset

In [ ]:
data = load_dataset("csv", data_files="midjourney_prompt_dataset.csv")

In [ ]:
data

In [ ]:
data["train"][0]

In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["User"]}
<assistant>: {data_point["Prompt"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

In [ ]:
data

# Finetune the model

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

# Save trained model

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "jzdesign/midjourney-falcon-7b"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

# Run the finetuned model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>: midjourney prompt for a boy running in the snow
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))